In [4]:
import polars as pl
import io
import zipfile
from pathlib import Path
from tempfile import TemporaryDirectory, TemporaryFile
from tqdm import tqdm
import re

In [47]:
pattern=r"(?<=국토교통부_)(.*)(?=\+\()"

split_characters = r"[_+() ]"


for file in Path("data/").glob("국토교통부_*.zip"):
    substrings = re.split(split_characters, file.name)
    name_segments = substrings[1:3]
    print(name_segments)  # i.e. 건축인허가, 가설건축물

['건축인허가', '가설건축물']
['건축인허가', '공작물관리대장']
['건축인허가', '기본개요']
['건축인허가', '대수선']
['건축인허가', '대지위치']
['건축인허가', '도로대장']
['건축인허가', '동별개요']
['건축인허가', '부설주차장']
['건축인허가', '오수정화시설']
['건축인허가', '전유공용면적']
['건축인허가', '주차장']
['건축인허가', '주택유형']
['건축인허가', '지역지구구역']
['건축인허가', '철거멸실관리대장']
['건축인허가', '층별개요']
['건축인허가', '호별개요']
['건축인허가', '호별전유공용면적']
['주택인허가', '관리공동부대복리시설']
['주택인허가', '관리공동형별개요']
['주택인허가', '기본개요']
['주택인허가', '대지위치']
['주택인허가', '동별개요']
['주택인허가', '복리분양시설']
['주택인허가', '부대시설']
['주택인허가', '부설주차장']
['주택인허가', '오수정화시설']
['주택인허가', '전유공용면적']
['주택인허가', '주차장']
['주택인허가', '지역지구구역']
['주택인허가', '층별개요']
['주택인허가', '행위개요']
['주택인허가', '행위호전유면적']
['주택인허가', '호별개요']


In [58]:
def get_polars_type(dtype_openeais: str):
    split_characters = r"[(,)]"

    substrings = re.split(split_characters, dtype_openeais)

    if substrings[0] in ["VARCHAR", "CHAR"]:
        return pl.Utf8
    elif substrings[0] in ["NUMERIC"]:
        if "," in dtype_openeais:
            return pl.Float64
        else:
            return pl.Int64
    else:
        raise


def get_schema_polars(path=""):
    if path:
        try:
            df = pl.read_csv(path, has_header=False)
            df = df.with_columns(df[:, 1].apply(get_polars_type))
            return df
        except:
            print("error", path)
    else:
        raise("no path")

In [59]:
print(Path("data/schema")/("_".join(["schema"] + name_segments) + ".csv"))

df_schema = get_schema_polars(Path("data/schema")/("_".join(["schema"] + name_segments) + ".csv"))
print(df_schema)

data\schema\schema_주택인허가_호별개요.csv
shape: (22, 2)
┌───────────────────┬──────────┐
│ column_1          ┆ column_2 │
│ ---               ┆ ---      │
│ str               ┆ object   │
╞═══════════════════╪══════════╡
│ 관리_호별_명세_PK ┆ Utf8     │
│ 관리_동별_개요_PK ┆ Utf8     │
│ 대지_위치         ┆ Utf8     │
│ 건물_명           ┆ Utf8     │
│ …                 ┆ …        │
│ 평형_구분_명      ┆ Utf8     │
│ 변경_구분_코드    ┆ Utf8     │
│ 변경_구분_코드_명 ┆ Utf8     │
│ 생성_일자         ┆ Utf8     │
└───────────────────┴──────────┘


In [63]:
df_schema[:, 1].to_list()

[Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Int64,
 Utf8,
 Utf8,
 Int64,
 Utf8,
 Utf8,
 Utf8,
 Utf8,
 Utf8]

In [66]:
df = pl.read_csv(
    "data/mart_jty_04.txt",
    has_header=False,
    separator="|",
    new_columns=df_schema[:, 0].to_list(),
    dtypes=df_schema[:, 1].to_list(),
    n_rows=10,
    encoding="cp949",
)


ComputeError: invalid utf-8 sequence in csv

In [60]:
with open("data/mart_jty_04.txt") as f:
    for i, line in enumerate(f):
        if i >=5:
            break
        print(line)

26410-100003037|26410-100004542|부산광역시 금정구 장전동 521번지|장전동 금정산 SK VIEW|26410|10800|0|0521|||||102|13|20|지상|0|1304|160|||20100828

26410-100003038|26410-100004542|부산광역시 금정구 장전동 521번지|장전동 금정산 SK VIEW|26410|10800|0|0521|||||102|14|20|지상|0|1401|183|||20100828

26410-100003039|26410-100004542|부산광역시 금정구 장전동 521번지|장전동 금정산 SK VIEW|26410|10800|0|0521|||||102|14|20|지상|0|1402|183|||20100828

26410-100003040|26410-100004542|부산광역시 금정구 장전동 521번지|장전동 금정산 SK VIEW|26410|10800|0|0521|||||102|14|20|지상|0|1403|160|||20100828

26410-100003041|26410-100004542|부산광역시 금정구 장전동 521번지|장전동 금정산 SK VIEW|26410|10800|0|0521|||||102|14|20|지상|0|1404|160|||20100828

